# Punto 01 del Informe de Machine Learning

Importaciones

In [1]:
import pandas as pd
import numpy as np
import warnings
import sklearn

1. Leer los datos del archivo titanic.csv.
Si el archivo se encuentra en su carpeta, corra el primer bloque, de no ser así, ejecute el segundo.

In [2]:
direccion = ("./data/titanic.csv")
datos = pd.read_csv(direccion, sep=";")

datos.columns = ["survived", "Name", "pclass", "sex", "age", "sibsp", "parch", "fare", "who", "embark_town", "alone"]

In [3]:
url = ("https://raw.githubusercontent.com/karenmoreno17/Informe-Machine-Learning/main/titanic.csv")
datos = pd.read_csv(url, sep=";")

datos.columns = ["survived", "Name", "pclass", "sex", "age", "sibsp", "parch", "fare", "who", "embark_town", "alone"]

2. Eliminación del atributo Name.

In [4]:
datos.drop(["Name"], axis = 1, inplace = True)

3. Selección del 80% de datos para entrenar, y el 20% restante para las pruebas.

In [5]:
from sklearn.model_selection import train_test_split 

n = len(datos)
datosEntreno = int(n * 0.8) #El 80% de los datos para el entreno y el 20% restante para las pruebas.
datosPruebas = n - datosEntreno
print(n, datosEntreno, datosPruebas) # Número de datos, datos de entreno y datos de pruebas
train_data, test_data = sklearn.model_selection.train_test_split(datos, train_size = datosEntreno, test_size = datosPruebas)

891 712 179


In [6]:
train_data.shape

(712, 10)

In [7]:
train_data.head()

,survived,pclass,sex,age,sibsp,parch,fare,who,embark_town,alone
419,0,3,female,10.0,0,2,24.1500,child,Southampton,no
384,0,3,male,NaN,0,0,7.8958,man,Southampton,yes
217,0,2,male,42.0,1,0,27.0000,man,Southampton,no
545,0,1,male,64.0,0,0,26.0000,man,Southampton,yes
530,1,2,female,2.0,1,1,26.0000,child,Southampton,no


4. Normalizar los datos.

In [8]:
# Hasta el momento, tenemos las siguientes variables:

# Variables cuantitativas (de tipo entero).
datos.select_dtypes(include = ['int64']).describe()

,survived,pclass,sibsp,parch
count,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.523008,0.381594
std,0.486592,0.836071,1.102743,0.806057
min,0.000000,1.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,0.000000
50%,0.000000,3.000000,0.000000,0.000000
75%,1.000000,3.000000,1.000000,0.000000
max,1.000000,3.000000,8.000000,6.000000


In [9]:
# Variables cuantitativas (de tipo flotante).
datos.select_dtypes(include = ['float64']).describe()

,age,fare
count,714.000000,891.000000
mean,29.699118,32.204208
std,14.526497,49.693429
min,0.420000,0.000000
25%,20.125000,7.910400
50%,28.000000,14.454200
75%,38.000000,31.000000
max,80.000000,512.329200


In [10]:
# Y variables categóricas (de tipo object).
datos.select_dtypes(include = ['object']).describe()

,sex,who,embark_town,alone
count,891,891,889,891
unique,2,3,3,2
top,male,man,Southampton,yes
freq,577,537,644,537


In [11]:
# Tenemos que crear los pipelines para cada las variables cuantitativas y las categóricas.

# ============================================================================================================================================================
# Pipeline para las variables cuantitativas.
# ============================================================================================================================================================

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Tenemos 6 variables cuantitativas
varCuantitativas = ['pclass', 'sibsp', 'parch', 'age', 'fare']

cuantiPipeline = Pipeline([
        ("imputer", SimpleImputer(strategy = "median")),
        ("scaler", StandardScaler()) 
    
    ])

In [12]:
# ============================================================================================================================================================
# Pipeline para las variables categóricas.
# ============================================================================================================================================================

from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# 6 atributos categóricos
varCategoricas = ['sex', 'who', 'embark_town', 'alone']

catePipeline = Pipeline([
        ("imputer", SimpleImputer(strategy = "most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse_output = False))
    ])

In [13]:
# ============================================================================================================================================================
# Pipeline completo.
# ============================================================================================================================================================

from sklearn.compose import ColumnTransformer

pipelineCompleto = ColumnTransformer([
    ("cuanti", cuantiPipeline, varCuantitativas),
    ("cate", catePipeline, varCategoricas), 
])

In [14]:
X_train = pipelineCompleto.fit_transform(train_data)

In [15]:
X_train.shape

(712, 15)

In [16]:
'''
Después de codificar y normalizar los datos, obtenemos 16 neuronas, de las cuales 5 corresponden a la variables cuantitativas,
y 10 (2 + 3 + 3 + 2) de las variables categóricas. 
'''

# Asignamos la variable dependiente, que para esta ocasión, es "survived"
Y_train = train_data['survived']
Y_train

419    0
384    0
217    0
545    0
530    1
      ..
525    0
653    1
320    0
627    1
126    0
Name: survived, Length: 712, dtype: int64

5. Construcción de 5 redes neuronales.

In [ ]:
# ============================================================================================================================================================
# Primera red neuronal.
# ============================================================================================================================================================
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

primeraRed = MLPClassifier(activation = 'identity', solver = 'lbfgs', alpha = 1e-5, hidden_layer_sizes = (6), random_state = 123)
primeraRed.fit(X_train, Y_train)
scoreRed01 = cross_val_score(primeraRed, X_train, Y_train, cv = 10, scoring='accuracy')
print(scoreRed01)
scoreRed01.mean()

In [ ]:
# ============================================================================================================================================================
# Segunda red neuronal.
# ============================================================================================================================================================
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

segundaRed = MLPClassifier(activation = 'tanh', solver = 'adam', alpha = 1e-5, hidden_layer_sizes = (6, 4, 2), random_state = 123)
segundaRed.fit(X_train, Y_train)
scoreRed02 = cross_val_score(segundaRed, X_train, Y_train, cv = 10, scoring='accuracy')
print(scoreRed01)
scoreRed02.mean()

In [ ]:
# ============================================================================================================================================================
# Tercera red neuronal.
# ============================================================================================================================================================
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

terceraRed = MLPClassifier(activation = 'tanh', solver = 'lbfgs', alpha = 1e-5, hidden_layer_sizes = (2, 3, 3), random_state = 123)
terceraRed.fit(X_train, Y_train)
scoreRed03 = cross_val_score(terceraRed, X_train, Y_train, cv = 10, scoring='accuracy')
print(scoreRed03)
scoreRed03.mean()

In [57]:
# ============================================================================================================================================================
# Cuarta red neuronal.
# ============================================================================================================================================================
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

cuartaRed = MLPClassifier(activation = 'relu', solver = 'lbfgs', alpha = 1e-5, hidden_layer_sizes = (5, 5, 5), random_state = 123)
cuartaRed.fit(X_train, Y_train)
scoreRed04 = cross_val_score(cuartaRed, X_train, Y_train, cv = 10, scoring='accuracy')
print(scoreRed04)
scoreRed04.mean()

c:\Users\Karen Daniela\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\Karen Daniela\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\Karen Daniela\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_netw

[0.81944444 0.75       0.76056338 0.84507042 0.78873239 0.83098592
 0.77464789 0.84507042 0.85915493 0.81690141]


c:\Users\Karen Daniela\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.8090571205007825

In [58]:
# ============================================================================================================================================================
# Quinta red neuronal.
# ============================================================================================================================================================
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

quintaRed = MLPClassifier(activation = 'logistic', solver = 'lbfgs', alpha = 1e-5, hidden_layer_sizes = (1, 5), random_state = 123)
quintaRed.fit(X_train, Y_train)
scoreRed05 = cross_val_score(quintaRed, X_train, Y_train, cv = 10, scoring='accuracy')
print(scoreRed05)
scoreRed05.mean()

[0.77777778 0.80555556 0.78873239 0.84507042 0.78873239 0.87323944
 0.78873239 0.83098592 0.88732394 0.81690141]


0.8203051643192488